## 問題集: 
使用內建因子時，出現以下 error message: <br>

ValueError: Bin edges must be unique: array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]). You can drop duplicate edges by setting the 'duplicates' kwarg

## 解決方法:

將日期設定為交易日之間。

# 導入價料資料

In [1]:
import os
os.environ["TEJAPI_BASE"] = "https://api.tej.com.tw"
os.environ["TEJAPI_KEY"] = "your key"
os.environ["ticker"] = "2330 2337 1101 2409 1301 1304 1315 1321 1504 2049 1540 1535 1517 2340 2303 3016 4967 3035 6515 3189"
os.environ['mdate'] = "20210104 20230708"
!zipline ingest -b tquant

Merging daily equity files:


[2023-09-04 10:38:07.046655] INFO: zipline.data.bundles.core: Ingesting tquant.


In [2]:
from zipline.data import bundles
bundle_name = 'tquant'
bundle = bundles.load(bundle_name)

# 問題

計算每家企業每日平均交易金額 (Average Dollar Volume)。因為 TQuant 在 2021/01/04(一) 時會導入 2021/01/03(日) 的資料，然而 1/3 為非交易日故沒有資料，造成 ValueError: Bin edges must be unique: array([nan, nan, nan, nan, nan]).
You can drop duplicate edges by setting the 'duplicates' kwarg。

In [3]:
from zipline.pipeline import Pipeline
from zipline.pipeline.factors import AverageDollarVolume
from zipline.utils.calendar_utils import get_calendar
from zipline.pipeline.data import EquityPricing
from zipline.api import attach_pipeline, pipeline_output
from zipline import run_algorithm
import pandas as pd

def make_pipeline():
    return Pipeline(
        columns = {
            "Deciles" : AverageDollarVolume(window_length = 2).quartiles()
        }
    )
def initialize(context):
    attach_pipeline(make_pipeline(), "my_strate")
    
def handle_data(context, data):
    outs = pipeline_output("my_strate")
    print(outs)
    
def analyze(context, perf):
    pass

results = run_algorithm(
    start = pd.Timestamp("2021-01-04", tz = 'utc'),
    end = pd.Timestamp("2023-01-01", tz = 'utc'),
    initialize = initialize,
    handle_data = handle_data,
    analyze = analyze,
    capital_base = 1e6, 
    bundle = 'tquant',
    data_frequency='daily'
)

ValueError: Bin edges must be unique: array([nan, nan, nan, nan, nan]).
You can drop duplicate edges by setting the 'duplicates' kwarg

# 解決

將 start 改成 2021/01/05 。

In [7]:
def make_pipeline():
    return Pipeline(
        columns = {
            "Deciles" : AverageDollarVolume(window_length = 2).quartiles()
        }
    )
def initialize(context):
    attach_pipeline(make_pipeline(), "my_strate")
    
def handle_data(context, data):
    outs = pipeline_output("my_strate")
    print(outs)
    
def analyze(context, perf):
    pass

results = run_algorithm(
    start = pd.Timestamp("2022-01-05", tz = 'utc'),
    end = pd.Timestamp("2023-01-03", tz = 'utc'),
    initialize = initialize,
    handle_data = handle_data,
    analyze = analyze,
    capital_base = 1e6, 
    bundle = 'tquant',
    data_frequency='daily'
)

                   Deciles
Equity(0 [1101])         2
Equity(1 [1301])         2
Equity(2 [1304])         1
Equity(3 [1315])         0
Equity(4 [1321])         0
Equity(5 [1504])         1
Equity(6 [1517])         0
Equity(7 [1535])         0
Equity(8 [1540])         0
Equity(9 [2049])         1
Equity(10 [2303])        3
Equity(11 [2330])        3
Equity(12 [2337])        2
Equity(13 [2340])        3
Equity(14 [2409])        3
Equity(15 [3016])        2
Equity(16 [3035])        3
Equity(17 [3189])        2
Equity(18 [4967])        1
Equity(19 [6515])        1
                   Deciles
Equity(0 [1101])         2
Equity(1 [1301])         2
Equity(2 [1304])         1
Equity(3 [1315])         0
Equity(4 [1321])         0
Equity(5 [1504])         1
Equity(6 [1517])         0
Equity(7 [1535])         0
Equity(8 [1540])         0
Equity(9 [2049])         2
Equity(10 [2303])        3
Equity(11 [2330])        3
Equity(12 [2337])        1
Equity(13 [2340])        3
Equity(14 [2409])        2
E

C:\Users\Admin\anaconda3\envs\tquant\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
C:\Users\Admin\anaconda3\envs\tquant\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(
C:\Users\Admin\anaconda3\envs\tquant\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
C:\Users\Admin\anaconda3\envs\tquant\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(
C:\Users\Admin\anaconda3\envs\tquant\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
C:\Users\Admin\anaconda3\envs\tquant\lib\site-packages\empyrical\stats.py:706: Runti

                   Deciles
Equity(0 [1101])         2
Equity(1 [1301])         2
Equity(2 [1304])         1
Equity(3 [1315])         0
Equity(4 [1321])         0
Equity(5 [1504])         1
Equity(6 [1517])         0
Equity(7 [1535])         0
Equity(8 [1540])         0
Equity(9 [2049])         1
Equity(10 [2303])        3
Equity(11 [2330])        3
Equity(12 [2337])        3
Equity(13 [2340])        2
Equity(14 [2409])        2
Equity(15 [3016])        2
Equity(16 [3035])        3
Equity(17 [3189])        3
Equity(18 [4967])        1
Equity(19 [6515])        1
                   Deciles
Equity(0 [1101])         2
Equity(1 [1301])         2
Equity(2 [1304])         1
Equity(3 [1315])         0
Equity(4 [1321])         0
Equity(5 [1504])         1
Equity(6 [1517])         0
Equity(7 [1535])         0
Equity(8 [1540])         0
Equity(9 [2049])         1
Equity(10 [2303])        3
Equity(11 [2330])        3
Equity(12 [2337])        3
Equity(13 [2340])        2
Equity(14 [2409])        2
E

C:\Users\Admin\anaconda3\envs\tquant\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
C:\Users\Admin\anaconda3\envs\tquant\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(
C:\Users\Admin\anaconda3\envs\tquant\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
C:\Users\Admin\anaconda3\envs\tquant\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(
C:\Users\Admin\anaconda3\envs\tquant\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
C:\Users\Admin\anaconda3\envs\tquant\lib\site-packages\empyrical\stats.py:706: Runti

                   Deciles
Equity(0 [1101])         2
Equity(1 [1301])         2
Equity(2 [1304])         1
Equity(3 [1315])         0
Equity(4 [1321])         0
Equity(5 [1504])         1
Equity(6 [1517])         0
Equity(7 [1535])         0
Equity(8 [1540])         0
Equity(9 [2049])         2
Equity(10 [2303])        3
Equity(11 [2330])        3
Equity(12 [2337])        2
Equity(13 [2340])        1
Equity(14 [2409])        3
Equity(15 [3016])        2
Equity(16 [3035])        3
Equity(17 [3189])        3
Equity(18 [4967])        1
Equity(19 [6515])        1
                   Deciles
Equity(0 [1101])         3
Equity(1 [1301])         2
Equity(2 [1304])         1
Equity(3 [1315])         0
Equity(4 [1321])         0
Equity(5 [1504])         1
Equity(6 [1517])         0
Equity(7 [1535])         0
Equity(8 [1540])         0
Equity(9 [2049])         2
Equity(10 [2303])        3
Equity(11 [2330])        3
Equity(12 [2337])        2
Equity(13 [2340])        1
Equity(14 [2409])        2
E

...

                   Deciles
Equity(0 [1101])         3
Equity(1 [1301])         2
Equity(2 [1304])         1
Equity(3 [1315])         0
Equity(4 [1321])         0
Equity(5 [1504])         1
Equity(6 [1517])         1
Equity(7 [1535])         0
Equity(8 [1540])         0
Equity(9 [2049])         2
Equity(10 [2303])        3
Equity(11 [2330])        3
Equity(12 [2337])        2
Equity(13 [2340])        1
Equity(14 [2409])        2
Equity(15 [3016])        2
Equity(16 [3035])        3
Equity(17 [3189])        3
Equity(18 [4967])        0
Equity(19 [6515])        1
                   Deciles
Equity(0 [1101])         3
Equity(1 [1301])         2
Equity(2 [1304])         1
Equity(3 [1315])         0
Equity(4 [1321])         0
Equity(5 [1504])         1
Equity(6 [1517])         1
Equity(7 [1535])         0
Equity(8 [1540])         0
Equity(9 [2049])         2
Equity(10 [2303])        3
Equity(11 [2330])        3
Equity(12 [2337])        2
Equity(13 [2340])        1
Equity(14 [2409])        2
E

C:\Users\Admin\anaconda3\envs\tquant\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
C:\Users\Admin\anaconda3\envs\tquant\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(
C:\Users\Admin\anaconda3\envs\tquant\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
C:\Users\Admin\anaconda3\envs\tquant\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(
C:\Users\Admin\anaconda3\envs\tquant\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
C:\Users\Admin\anaconda3\envs\tquant\lib\site-packages\empyrical\stats.py:706: Runti

                   Deciles
Equity(0 [1101])         2
Equity(1 [1301])         2
Equity(2 [1304])         1
Equity(3 [1315])         0
Equity(4 [1321])         0
Equity(5 [1504])         1
Equity(6 [1517])         1
Equity(7 [1535])         0
Equity(8 [1540])         0
Equity(9 [2049])         2
Equity(10 [2303])        3
Equity(11 [2330])        3
Equity(12 [2337])        2
Equity(13 [2340])        1
Equity(14 [2409])        3
Equity(15 [3016])        1
Equity(16 [3035])        3
Equity(17 [3189])        3
Equity(18 [4967])        0
Equity(19 [6515])        2
                   Deciles
Equity(0 [1101])         3
Equity(1 [1301])         2
Equity(2 [1304])         1
Equity(3 [1315])         0
Equity(4 [1321])         0
Equity(5 [1504])         1
Equity(6 [1517])         1
Equity(7 [1535])         0
Equity(8 [1540])         0
Equity(9 [2049])         2
Equity(10 [2303])        3
Equity(11 [2330])        3
Equity(12 [2337])        2
Equity(13 [2340])        1
Equity(14 [2409])        2
E

                   Deciles
Equity(0 [1101])         2
Equity(1 [1301])         3
Equity(2 [1304])         1
Equity(3 [1315])         0
Equity(4 [1321])         0
Equity(5 [1504])         1
Equity(6 [1517])         1
Equity(7 [1535])         0
Equity(8 [1540])         0
Equity(9 [2049])         2
Equity(10 [2303])        3
Equity(11 [2330])        3
Equity(12 [2337])        2
Equity(13 [2340])        1
Equity(14 [2409])        2
Equity(15 [3016])        1
Equity(16 [3035])        3
Equity(17 [3189])        3
Equity(18 [4967])        0
Equity(19 [6515])        2
                   Deciles
Equity(0 [1101])         2
Equity(1 [1301])         3
Equity(2 [1304])         1
Equity(3 [1315])         0
Equity(4 [1321])         0
Equity(5 [1504])         1
Equity(6 [1517])         1
Equity(7 [1535])         0
Equity(8 [1540])         0
Equity(9 [2049])         2
Equity(10 [2303])        3
Equity(11 [2330])        3
Equity(12 [2337])        2
Equity(13 [2340])        1
Equity(14 [2409])        2
E

C:\Users\Admin\anaconda3\envs\tquant\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
C:\Users\Admin\anaconda3\envs\tquant\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(
C:\Users\Admin\anaconda3\envs\tquant\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
C:\Users\Admin\anaconda3\envs\tquant\lib\site-packages\empyrical\stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(
C:\Users\Admin\anaconda3\envs\tquant\lib\site-packages\empyrical\stats.py:797: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
C:\Users\Admin\anaconda3\envs\tquant\lib\site-packages\empyrical\stats.py:706: Runti